# Spørring mot verksregisteret

Oddrun Ohren 

Lars G Johnsen 

## Kode

In [1]:
import json
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import dhlab.text as dh
import dhlab.api.dhlab_api as dhapi

In [2]:
%%HTML
<style>
p {
    font-size:1.3em;
};
h1, h2, h3, h4 {
  background-color: black;
  color: white;
}
</style>

In [3]:
def query_verksregister(forfatter="", tittel="", page = 1):
    """spør mot verksregisteret 
    api for webgrensesnittet https://livedata.bibsys.no/verksregister/"""
    
    # sett opp query
    params = {
        "query" : f"(forfatter:{forfatter}) AND (tittel:{tittel})",
        "page": page
    }
    
    #utfør spørring
    res = requests.get("https://livedata.bibsys.no/works-registry-api/search/searchWorksbyTitleAndCreator", params = params)
    if res.status_code == 200:
        res = res.text
    else:
        res = None
    
    return res

In [4]:
def find_works(forfatter="", tittel=""):
    """finn verk i registeret
    returner dem som pandas dataramme"""
    
    query_res = query_verksregister(forfatter, tittel)
    if query_res is None:
        return None
    soup = BeautifulSoup(query_res)
    n = soup.find("p")
    struct = json.loads(n.text)
    res = pd.DataFrame(struct['results'])
    return res

In [5]:
def works(identifier):
    """Hent ut data om et verk fra identifikatoren"""
    
    res = requests.get(f"https://livedata.bibsys.no/works-registry-api/biblio/byWork/{identifier}")
    try:
        r = json.loads(res.text)
    except:
        r = {}
    return r

In [6]:
def url_to_urn(url):
    """Pell ut URN fra en URL"""

    a = re.findall("URN.*", url)
    if a != []:
        res = a[0]
    else:
        res = ''
    return res

In [7]:
def urns_for_works(works_id):
    """Fra en verks-ID hent ut alle URNer
    param:
        works_id a work identifier - from """
    
    graph = works(works_id)['@graph']
    res = [[url_to_urn(x['id']) for x in i['bibsys:url_to_content']] for i in graph if 'bibsys:url_to_content' in i]
    result = [x for y in res for x in y if x != '']
    return result

# Let etter en bestemt bok

Det her kan sikkert generaliseres på en måte, men tenker at det er litt manuelt arbeid i å konstruere korpuset.

In [8]:
result = find_works("dante", "inferno")

Søket gir en dataramme, ideelt inneholder en en rad, der første rad er verksidentiteten.

In [9]:
result

,id,title,language,quality,created,manifested,creators,types,deleted
0,bd400f20b17f4bbbf0fdc55f0f4f5f6de29f67ca96ec61...,Inferno :den guddommelige komedie,nob,kat2,2018-07-24T11:43:30.000+0000,2017-01-01T00:00:00.000+0000,"[{'name': 'Dante Alighieri', 'birth': 1265, 'd...","[{'id': 'V1000', 'labels': {'no': 'Språkbasert...",False
1,ad1b22cc64d728a90334f2d825c09bf57debb5d637a073...,"La divina commedia, Inferno",nno,kat2,2018-07-24T11:12:44.000+0000,1993-01-01T00:00:00.000+0000,"[{'name': 'Dante Alighieri', 'birth': 1265, 'd...","[{'id': 'V1000', 'labels': {'no': 'Språkbasert...",False


Finn alle URNer for verket

In [12]:
result.id[0]

'bd400f20b17f4bbbf0fdc55f0f4f5f6de29f67ca96ec61c1511bc526bdf02a3f'

In [17]:
urns = urns_for_works(result.id[1])

In [18]:
urns

['URN:NBN:no-nb_digibok_2008011601096']

Let opp URN-ene i dhlab, og sorter dem på år. Datarammen gir det som skal til for å konstruere parallelltekster. Det er en del titler som inneholder flere tekster, så det må sjekkes. Ellers kan tekster velges ut mer eller mindre automatisk fra årsspenn, og knyttes til språkendringsperioder som 1907, 1917, 193x, 19.. og flere.

In [19]:
dhapi.get_metadata(urns).sort_values(by = 'year')

,dhlabid,title,authors,urn,oaiid,sesamid,isbn10,city,timestamp,year,publisher,langs,subjects,ddc,genres,literaryform,doctype
0,100460043,Helvetet,"Dante Alighieri / Ulleland , Magnus",URN:NBN:no-nb_digibok_2008011601096,oai:nb.bibsys.no:999322108304702202,e1272051d2068047c58b859b6f66e350,8205219214,Oslo,19930101,1993,Gyldendal,nno / ita,italia / litteratur / italiensk,851,poetry,Skjønnlitteratur,digibok


In [ ]:
amtmann_korpus = dh.urnlist

In [ ]:
amtmann_korpus.corpus